In [2]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, because Keras may or maynot contain the features discussed here depending upon when you read this article, until the keras_preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [30]:
df=pd.read_csv("labels.csv",names=["img","prob","label"])

In [29]:
df["label"].values

array(['mono', 'mono', 'mono', ..., 'poly', 'poly', 'poly'], dtype=object)

In [31]:
columns=["label"]
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="./",
x_col="img",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["mono","poly"])
valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[1800:1900],
directory="./",
x_col="img",
y_col="label",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["mono","poly"])
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[1900:],
directory="./",
x_col="img",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None)

Found 1541 validated image filenames belonging to 2 classes.
Found 79 validated image filenames belonging to 2 classes.
Found 724 validated image filenames.


In [32]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [33]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
48/48 [==============================] - 222s 5s/step - loss: 1.0885 - acc: 0.5173 - val_loss: 0.7236 - val_acc: 0.0547
Epoch 2/5
48/48 [==============================] - 200s 4s/step - loss: 0.6480 - acc: 0.6332 - val_loss: 0.7138 - val_acc: 0.0638
Epoch 3/5
48/48 [==============================] - 256s 5s/step - loss: 0.5688 - acc: 0.7170 - val_loss: 0.8137 - val_acc: 0.0638
Epoch 4/5
48/48 [==============================] - 197s 4s/step - loss: 0.4627 - acc: 0.7879 - val_loss: 0.5455 - val_acc: 0.9688
Epoch 5/5
48/48 [==============================] - 197s 4s/step - loss: 0.3514 - acc: 0.8737 - val_loss: 0.5461 - val_acc: 0.9043


In [34]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

724/724 [==============================] - 65s 89ms/step


In [37]:
pred_bool = (pred >0.5)
predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)